In [1]:
# model training
import h5py 
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, Input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, UpSampling2D
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
import numpy as np
from sklearn.metrics import confusion_matrix
import gc
import csv
from keras import backend as K
import matplotlib.pyplot as plt
import os

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

tf.config.optimizer.set_experimental_options({'layout_optimizer': False})        
       
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')    


weight_sens=15
batch_sens=5


with h5py.File('NSIDC_oversampled.h5', "r") as f: #192 x 192 x 1000 oversampled scenes
    X = f['sic'][()] #values between 0 and 1

with h5py.File('polynyaloc_oversampled.h5', "r") as g: #corresponding 192 x 192 x 1000 masks
    Y = g['polynyaloc'][()] #values of 0 (no polyna) or 1 (polynya)
    
    
def unet(input_shape=(192, 192, 1)):
    inputs = tf.keras.Input(shape=input_shape)  # (H, W, 1)

    # Encoder
    c1 = layers.Conv2D(16, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D(pool_size=(2, 2))(c1)

    c2 = layers.Conv2D(32, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D(pool_size=(2, 2))(c2)

    # Bottleneck
    c3 = layers.Conv2D(64, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(64, 3, activation='relu', padding='same')(c3)

    # Decoder
    u4 = layers.Conv2DTranspose(32, kernel_size=2, strides=2, padding='same')(c3)
    u4 = layers.Concatenate()([u4, c2])
    c4 = layers.Conv2D(32, 3, activation='relu', padding='same')(u4)
    c4 = layers.Conv2D(32, 3, activation='relu', padding='same')(c4)

    u5 = layers.Conv2DTranspose(16, kernel_size=2, strides=2, padding='same')(c4)
    u5 = layers.Concatenate()([u5, c1])
    c5 = layers.Conv2D(16, 3, activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(16, 3, activation='relu', padding='same')(c5)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c5)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model


def weighted_bce(pos_weight=99.0): 
    def loss(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        loss = - (pos_weight * y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
        return tf.reduce_mean(loss)
    return loss

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_binary = tf.cast(y_pred > self.threshold, tf.float32)
        self.precision.update_state(y_true, y_pred_binary, sample_weight)
        self.recall.update_state(y_true, y_pred_binary, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + 1e-8)

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

        
met_f1=np.zeros(100)
met_precis=np.zeros(100)
#storing true positives, true negatives, false positives and false negatives in separate arrays
TP=np.zeros(100) 
TN=np.zeros(100)
FP=np.zeros(100)
FP=np.zeros(100)
        
for i in range(100):    
    X_tr, X0, y_tr, y0 = train_test_split(X, Y, test_size=0.3)
    X_te, X_va, y_te, y_va = train_test_split(X0, y0, test_size=0.5)
    del X0,y0

    model = unet(input_shape=(192, 192, 1) )
    model.compile(optimizer='adam', loss=weighted_bce(pos_weight=weight_sens), metrics=[F1Score(), tf.keras.metrics.Precision()])#, tf.keras.metrics.Recall()])
    model.fit(X_tr, y_tr, batch_size=batch_sens, verbose=0, validation_data=(X_va, y_va), epochs=30)
    
    model.save('CNN_model/CNN2D_wbce_w'+str(weight_sens)+'_B'+str(batch_sens)+'_'+str(i)+'.keras')

    Y_pred=model.predict(X_va, verbose=0)
    junk1=np.round(Y_pred) #turns prediction into binary 0 or 1 mask
    
    met_f1[i]=f1_score(y_true=y_va.flatten(), y_pred=junk1.flatten())
    met_precis[i]=precision_score(y_true=y_va.flatten(), y_pred=junk1.flatten())    
    jconf=confusion_matrix(y_true=y_va.flatten(), y_pred=junk1.flatten())
    met_TN[i]=jconf[0,0]
    met_FP[i]=jconf[0,1]
    met_FN[i]=jconf[1,0]
    met_TP[i]=jconf[1,1]
    
    del model, Y_pred, junk1, jconf
    tf.keras.backend.clear_session()
    gc.collect()
    
outfilename="CNN_model/CNN2D_wbce_w"+str(weight_sens)+"_B"+str(batch_sens)+".csv"
fields = ['F1score','Precision','TN','FP','FN','TP']   
with open(str(outfilename), 'w') as f:
    f.truncate()
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(fields)
    writer.writerows(zip(met_f1,met_precis,met_TN,met_FP,met_FN,met_TP)) 
    

2025-05-20 13:04:54.566427: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-20 13:04:55.224282: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-20 13:05:04.558558: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler f

/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `rese

/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `rese

/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `rese

In [ ]:
#using the best model on the original, non oversampled dataset

weight_sens=15
batch_sens=5
i=91

with h5py.File('allNSIDC_fortesting.h5', "r") as f: #192 x 192 x 7300 original daily images
    X = f['sic'][()] 

X=np.where(np.isnan(X),1,X)        
model=keras.models.load_model('CNN_model/CNN2D_wbce_w'+str(weight_sens)+'_B'+str(batch_sens)+'_'+str(i)+'.keras',compile=False)
Y_pred=model.predict(X, verbose=0)
Y_pred=np.squeeze(Y_pred)

f1 = h5py.File("CNN2D_wbce_w"+str(weight_sens)+"_B"+str(batch_sens)+"_"+str(i)+".hdf5", "w")
dset1 = f1.create_dataset("SIC_pred", Y_pred.shape, dtype=np.float32, data=Y_pred)
f1.close()